
#### Introduction

Implementation of RNNs for sentence generation using Keras. The reference taken is [this](https://chunml.github.io/ChunML.github.io/project/Creating-Text-Generator-Using-Recurrent-Neural-Network/) website. The dataset use is Andrej Karpathy's [page](https://cs.stanford.edu/people/karpathy/char-rnn/)

First we will download the text if required and load the data as characters in the text

In [1]:
import os
from urllib.request import urlretrieve

def maybe_download(target_url, local_dir, local_file):
    if not os.path.exists(local_dir):
        os.mkdir(local_dir)
        
    complete_path = os.path.join(local_dir, local_file)
    if os.path.exists(complete_path):
        print('File %s exists, not downloading'%complete_path)
    else:
        print('Downloading %s'%target_url)
        urlretrieve(target_url, complete_path)
        print('File downloaded')
    return complete_path

url = 'https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt'
local_file = maybe_download(url, 'Shakespeare', 'shakespeare_input.txt')

with open(local_file, 'r') as f:
    data = f.read()
    
chars = list(set(data))
print('Number of unique characters are', len(chars))

File Shakespeare/shakespeare_input.txt exists, not downloading
Number of unique characters are 67


Create lookup and reverse lookup for vocabulary

In [9]:
char_to_ix = {c:ix for ix, c in enumerate(chars)}
ix_to_char = {ix:c for ix, c in enumerate(chars)}

Lets define some parameters for the network

In [33]:

seq_length = 50 #Number of previous characters (timesteps) to train on

vocab_size = len(chars)

hidden_dim = 500

batch_size = 50

num_layers = 2

generate_length = 500





Define input and output numpy arrays

In [28]:
import numpy as np

def build_input(input_text):
    batches = len(input_text) // seq_length
    X = np.zeros([batches, seq_length, vocab_size])
    Y = np.zeros([batches, seq_length, vocab_size])
    
    for i in range(0, batches):
        x_slice_text = input_text[(i * seq_length) : (i + 1) * seq_length]
        y_slice_text = input_text[(i * seq_length + 1) : ((i + 1) * seq_length) + 1]
        x_slice_text_ix = [char_to_ix[t] for t in x_slice_text]
        y_slice_text_ix = [char_to_ix[t] for t in y_slice_text]
        for j in range(seq_length):
            X[i, j, x_slice_text_ix[j]] = 1
            Y[i, j, y_slice_text_ix[j]] = 1            
            
    return X, Y

X, Y = build_input(data)

Method to generate the text

In [30]:
def generate_text(model, length, vocab_size, ix_to_char):
    ix = np.random.randint(vocab_size)
    y_char = [ix_to_char[ix]]
    X = np.zeros((1, length, vocab_size))
    for i in range(length):
        # appending the last predicted character to sequence
        X[0, i, :][ix] = 1
        print(ix_to_char[ix], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix])
    return ('').join(y_char)


Now lets define the model

In [32]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers.wrappers import TimeDistributed

model = Sequential()
model.add(LSTM(hidden_dim, input_shape=(None, vocab_size), return_sequences=True))
for i in range(num_layers - 1):
  model.add(LSTM(hidden_dim, return_sequences=True))
model.add(TimeDistributed(Dense(vocab_size)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")


Using TensorFlow backend.


In [ ]:
num_epoch = 0

while num_epoch < 10:
    print('\n\nEpoch: {}\n'.format(num_epoch))
    model.fit(X, Y, batch_size = batch_size, verbose = 1, epochs = 1)
    num_epoch += 1
    generate_text(model, generate_length, vocab_size, ix_to_char)



Epoch: 0

Epoch 1/1
55550/91466 [=================>............] - ETA: 18:49 - loss: 2.0615